In [1]:
import numpy as np               
import pandas as pd              # DataFrame
import matplotlib.pyplot as plt  # Visualization
import seaborn as sns            # Visualization






ModuleNotFoundError: No module named 'seaborn'

In [2]:
# set option, change the rows and cols max display 
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Import banking data
client = pd.read_csv('client.asc', sep=';')
account = pd.read_csv('account.asc', sep=';')
card = pd.read_csv('card.asc', sep=';')
disp = pd.read_csv('disp.asc', sep=';')


loan = pd.read_csv('loan.asc', sep=';')
district = pd.read_csv('district.asc', sep=';')


In [3]:

# Transform the birth day into year
client['birth_year'] = client['birth_number'].transform(lambda bn: int('19' + str(bn)[:2]))

# Age
client['age'] = 1999 - client['birth_year']

# Age group
client['age_group'] = client['age'] // 10 * 10

# Function to extract birth month and gender
def to_month_gender(birth_number):
    
    s = str(birth_number)
    birth_month = int(s[2:4])
    
    if birth_month > 50:
        gender = "F"
        birth_month = birth_month - 50
    else:
        gender = 'M'
        
    return pd.Series({'birth_month':birth_month, 'gender':gender})

client[['birth_month', 'gender']] = client['birth_number'].apply(to_month_gender)




In [4]:
# Merge the client, disp and loan tables
client_acc = pd.merge(client, disp, on='client_id', how='left')
client_loan = pd.merge(client_acc, loan, on='account_id', how='left')
client_loan.head()


,client_id,birth_number,district_id,birth_year,age,age_group,birth_month,gender,disp_id,account_id,type,loan_id,date,amount,duration,payments,status
0,1,706213,18,1970,29,20,12,F,1,1,OWNER,NaN,NaN,NaN,NaN,NaN,NaN
1,2,450204,1,1945,54,50,2,M,2,2,OWNER,4959.0,940105.0,80952.0,24.0,3373.0,A
2,3,406009,1,1940,59,50,10,F,3,2,DISPONENT,4959.0,940105.0,80952.0,24.0,3373.0,A
3,4,561201,5,1956,43,40,12,M,4,3,OWNER,NaN,NaN,NaN,NaN,NaN,NaN
4,5,605703,5,1960,39,30,7,F,5,3,DISPONENT,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:

###############################################################################
# Clean transaction table
###############################################################################
trans = pd.read_csv('trans.asc', sep=';', low_memory=False)
# def functions: convert to English (Refer to transaction table's remarks)
# using if-else, return NaN for unmatched values
def convert_trans_type_to_eng(x):
    if x == 'PRIJEM':
        return 'Credit'
    elif x == 'VYDAJ':
        return 'Withdrawal'
    else:
        return np.NaN
    
def convert_trans_op_to_eng(x):
    if x == 'VYBER KARTOU':
        return 'Credit card withdrawal'
    elif x == 'VKLAD':
        return 'Credit in cash'
    elif x == 'PREVOD Z UCTU':
        return 'Collection from another bank'
    elif x == 'VYBER':
        return 'Withdrawal in Cash'
    elif x == 'PREVOD NA UCET':
        return 'Remittance to another bank'    
    else:
        return np.NaN
    
def convert_trans_k_symbol_to_eng(x):
    if x == 'POJISTNE':
        return 'Insurance payment'
    elif x == 'SLUZBY':
        return 'Payment for statement'
    elif x == 'UROK':
        return 'Interest credited'
    elif x == 'SANKC. UROK':
        return 'Sanction interest if negative balance'
    elif x == 'SIPO':
        return 'Household'
    elif x == 'DUCHOD':
        return 'Old-age pension'  
    elif x == 'UVER':
        return 'Loan payment'      
    else:
        return np.NaN
    

In [6]:
# use map to apply above functions to type, operation, amount columns
trans['trans_type'] = trans['type'].map(convert_trans_type_to_eng)
trans['trans_operation'] = trans['operation'].map(convert_trans_op_to_eng)
trans['trans_k_symbol'] = trans['k_symbol'].map(convert_trans_k_symbol_to_eng)

# drop original columns after modified
trans=trans.drop(['type', 'operation','k_symbol'], axis=1)

# converts the date into a proper date format
trans['date'] = pd.to_datetime(trans['date'], format='%y%m%d')

# rename the column names as per remarks
trans = trans.rename(columns={'amount': 'trans_amount',
                              'balance':'balance_after_trans',
                              'bank':'trans_bank_partner',
                              'account':'trans_account_partner'})
# check the info and samples
print(trans.info())
print(trans.sample(5))

# export to csv
trans.to_csv('trans_cleaned.csv', header=True, sep=';') 
print('export completed!')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056320 entries, 0 to 1056319
Data columns (total 10 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   trans_id               1056320 non-null  int64         
 1   account_id             1056320 non-null  int64         
 2   date                   1056320 non-null  datetime64[ns]
 3   trans_amount           1056320 non-null  float64       
 4   balance_after_trans    1056320 non-null  float64       
 5   trans_bank_partner     273508 non-null   object        
 6   trans_account_partner  295389 non-null   float64       
 7   trans_type             1039654 non-null  object        
 8   trans_operation        873206 non-null   object        
 9   trans_k_symbol         521006 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 80.6+ MB
None
        trans_id  account_id       date  trans_amount  balance_after

In [7]:
###############################################################################
# Clean order table
###############################################################################
order = pd.read_csv('order.asc', sep=';')
# def function: convert to English (Refer to order table's remarks)
# use the same translation as transaction table to keep the integrity
# use if-else, return NaN for unmatched values
def convert_order_k_symbol_to_eng(x):
    if x == 'POJISTNE':
        return 'Insurance payment'
    elif x == 'SIPO':
        return 'Household'
    elif x == 'LEASING':
        return 'Leasing'
    elif x == 'UVER':
        return 'Loan payment'
    else:
        return np.NaN

In [8]:

# use map, apply to k_symbol
order['order_k_symbol'] = order['k_symbol'].map(convert_order_k_symbol_to_eng)
# drop original columns after modified
order=order.drop(['k_symbol'], axis=1)
# rename the column names as per remarks
order = order.rename(columns={'bank_to': 'order_recipient_bank',
                              'account_to':'order_recipient_account',
                              'amount':'order_amount'})
    
    # check the info and samples
print(order.info())
print(order.sample(5))

# export to csv
order.to_csv('order_cleaned.csv', header=True, sep=';') 
print('export completed!')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6471 entries, 0 to 6470
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 6471 non-null   int64  
 1   account_id               6471 non-null   int64  
 2   order_recipient_bank     6471 non-null   object 
 3   order_recipient_account  6471 non-null   int64  
 4   order_amount             6471 non-null   float64
 5   order_k_symbol           5092 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 303.5+ KB
None
      order_id  account_id order_recipient_bank  order_recipient_account  \
1618     31188        1223                   CD                 87986993   
5401     35718        4293                   GH                 89613037   
5402     35719        4293                   IJ                 75312357   
3440     33202        2568                   KL                 34637659   
3616     3339